# Algoritmo de desarrollo orientado a objetos y numpy
Se busca probar las funciones y scripts para obtener una mayor rapidez de ejecucion.
Se generan los algoritmos necesarios para el desarrollo del algoritmo que compare ambas estructuras
Basandose en la estructura como esta en start_oo.py

In [1]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
# herramientas para leer pdbs
import read_pdb_tools as rpt
# calculo de distancia
from scipy.spatial.distance import pdist, squareform
# libreria de tablas
import pandas as pd
# funciones de click generadas en pandas
import funciones_CLICK as fc
# iteradores
import itertools as it
# cuenta tiempo de ejecucion
import datetime
#distancia
from scipy.spatial import distance
#multiprocesamiento
import multiprocessing
from functools import partial

In [2]:
tiempo =  datetime.datetime.now()

In [3]:
# assert( len(sys.argv) > 1)
# lectura de archivo
file1 = 'pdbs/1xxa.pdb' # sys.argv[1]
file2 = 'pdbs/1tig.pdb' # sys.argv[2]

# numero de cliques, preguntar en el software para generalizarlo...
num_cliques = 3

# se define la estructura
pdb1 = rpt.PdbStruct(file1)
pdb2 = rpt.PdbStruct(file2)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)
pdb2.AddPdbData("%s" % file2)

# se obtienen los residuos que perteneces a la cadena de interes por default chain = 'A'
pdb11 = pdb1.GetResChain()
pdb22 = pdb2.GetResChain()

pdb1.Set_SS()
pdb2.Set_SS()

ss1 = fc.create_ss_table(pdb11)
ss2 = fc.create_ss_table(pdb22)

In [4]:
def get_df_distancias(ref):
    """Funcion para obtener el dataframe de distancias de cada proteina"""
    # se generan listas con coordenadas y numero de atomo
    coord = [res.GetAtom('CA').coord for res in ref]
    index = [res.resi for res in ref]

    # calcula distancia y regresa dataframe
    distancias = []
    # se calcula la distancia euclidiana entre cada atomo de carbon alfalfa
    for v in coord:
        distancia_un_atomo = []
        for av in coord:
            distancia = pdist(np.array([v, av]), metric='euclidean').item()
            distancia_un_atomo.append(distancia)
        distancias.append(distancia_un_atomo)

    # se genera la matriz de adyacencias para la red
    df_da = pd.DataFrame(index=index, columns=index, data=distancias)
    return(df_da, index)


df_distancias1, index1 = get_df_distancias(pdb11)
df_distancias2, index2 = get_df_distancias(pdb22)

In [6]:
# se generan cliques, tte devuleve dataframe con cliques de 3 y la lista de cliques sin partir
df_cliques1, cliques1 = fc.gen_3_cliques(df_distancias1, 'eso',dth=10, k=num_cliques)
print('**'*50)
df_cliques2, cliques2 = fc.gen_3_cliques(df_distancias2, 'eso',dth=10, k=num_cliques)
print('**'*50)

numero de cliques maximos encontrados: 158
numero de 3-cliques posibles: 1787
****************************************************************************************************
numero de cliques maximos encontrados: 246
numero de 3-cliques posibles: 2102
****************************************************************************************************


In [19]:
109*58

50576

In [16]:
cliques1.groupby('numero_elementos').count()[0]

numero_elementos
4     4 
5     12
6     42
7     58
8     26
9     13
10    3 
Name: 0, dtype: int64

In [112]:
def get_df_ca(list_of_residues):
    """Genera dataframe con la informacion necesaria para las siguientes funciones
    FALTA DOCUMENTAR ESTA COSA!!!!"""
    #crear df_ca
    atom_number = []
    atom_name = []
    residue_name = []
    residue_number = []
    coord = []
    for res in list_of_residues:
        for atom in res.atoms:
            atom_number.append(atom.atom_number)
            atom_name.append(atom.name)
            residue_name.append(res.resn)
            residue_number.append(res.resi)
            coord.append(atom.coord)

    df_atoms = pd.DataFrame(columns=['atom_number', 'atom_name', 'residue_name',
                                   'residue_number', 'vector'])
    df_atoms.atom_number = atom_number
    df_atoms.atom_name = atom_name
    df_atoms.residue_name = residue_name
    df_atoms.residue_number = residue_number
    df_atoms.vector = coord

    return(df_atoms)


# CREAR DF_atomos_CA #
df_atoms1 = get_df_ca(pdb11)
df_atoms2 = get_df_ca(pdb22)

In [113]:
# se le pega la estructura secundaria al dataframe de los cliques
# esto va a cambiar por que lo tiene que obtener del objeto residuo
# ya se crea en ss1 y no cuesta reevaluar si es mejor desde el residuo
# checar que es mas rapido si desde residuo o desde dataframe ss
df_cliques1 = fc.paste_SS(ss1, df_cliques1, num_cliques = num_cliques)
df_cliques2 = fc.paste_SS(ss2, df_cliques2, num_cliques = num_cliques)

In [114]:
# comparacion SSM #aqui se obtienen los candidatos posibles pasando el filtro de SS
candidatos_ss = fc.compare_SS(df_cliques1,df_cliques2, num_cliques=num_cliques)

In [115]:
# get coords of cliques
df_cliques1 = fc.get_coords_clique(df_atoms1, df_cliques1, num_cliques)
df_cliques2 = fc.get_coords_clique(df_atoms2, df_cliques2, num_cliques)

In [116]:
# baricentro clique
df_cliques1 = fc.baricenter_clique(df_cliques1, num_cliques)
df_cliques2 = fc.baricenter_clique(df_cliques2, num_cliques)

In [117]:
# vectores gorro
df_cliques1 = fc.center_vectors(df_cliques1, num_cliques)
df_cliques2 = fc.center_vectors(df_cliques2, num_cliques)

In [118]:
for i,j in enumerate(df_cliques1.columns):
    print(i,j)

0 0
1 1
2 2
3 ss_0
4 ss_1
5 ss_2
6 coord_clique_0
7 coord_clique_1
8 coord_clique_2
9 matriz_coordenadas
10 baricentro_clique
11 vec_gorro_0
12 vec_gorro_1
13 vec_gorro_2
14 vectores_gorro


In [119]:
idx_rmsd1, idx_rmsd2 = 3*num_cliques, 4*num_cliques+3
# print(list(range(idx_rmsd1,idx_rmsd2)))
# se pasan a numpy arrays para mayor rapidez
array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)] #del 9 al 15
array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]

In [120]:
#calculo del RMSD
print(len(candidatos_ss))

1705808


# Codigo que restringe candidatos

In [121]:
df_cliques1 = fc.get_distancia_promedio(num_cliques, df_cliques1)
df_cliques2 = fc.get_distancia_promedio(num_cliques, df_cliques2)

array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
array_dist_promedio2 = df_cliques2.values[:, -1]

In [122]:
limite_distancia_minima = 0.45
if num_cliques == 4:
    limite_distancia_minima = 0.9
if num_cliques == 5:
    limite_distancia_minima = 1.8
if num_cliques == 6:
    limite_distancia_minima = 3.6
if num_cliques == 7:
    limite_distancia_minima = 4.5
if num_cliques == 8:
    limite_distancia_minima = 8.0

candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
        array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
        array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

print(len(candidatos_ss))
print(len(candidatos_filter_dist))

1705808
253993


In [123]:
restriccion_rmsd = 0.15
if num_cliques == 4:
    restriccion_rmsd = 0.30
if num_cliques == 5:
    restriccion_rmsd = 0.60
if num_cliques == 6:
    restriccion_rmsd = 0.90
if num_cliques == 7:
    restriccion_rmsd = 1.50
if num_cliques == 8:
    restriccion_rmsd = 1.80

# Haciendo MUltiprocessing

In [124]:
p = multiprocessing.Pool(multiprocessing.cpu_count()-1)

In [125]:
%%time
rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,  
                    array_cliques1 = array_df_cliques1, 
                    array_cliques2 = array_df_cliques2, 
                    num_cliques = num_cliques), candidatos_filter_dist)

p.close()
p.join()

CPU times: user 4.91 s, sys: 92.7 ms, total: 5 s
Wall time: 13.9 s


In [126]:
len(rmsd_1)

253993

In [127]:
f1 = pd.DataFrame(rmsd_1)
f1.columns = ['rmsd','candidatos']
f1['cumple'] = np.where(f1.rmsd <= restriccion_rmsd,1,0)
f1['candidato_clique_1'] = f1.candidatos.str.get(0)
f1['candidato_clique_2'] = f1.candidatos.str.get(1)
f1.shape

(253993, 5)

In [128]:
print(f1[f1.cumple==1].cumple.sum())
f1 = f1[f1.cumple==1]

6907


In [129]:
df_cliques1.head()

,0,1,2,ss_0,ss_1,ss_2,coord_clique_0,coord_clique_1,coord_clique_2,matriz_coordenadas,baricentro_clique,vec_gorro_0,vec_gorro_1,vec_gorro_2,vectores_gorro,distancia_promedio
0,96,97,132,B,B,B,"[22.136, 75.657, 3.038]","[21.002, 74.191, 0.793]","[23.896, 66.017, 14.309]","[[22.136, 75.657, 3.038], [21.002, 74.191, 0.793], [23.896, 66.017, 14.309]]","[22.344666666666665, 71.955, 6.046666666666667]","[-0.2086666666666659, 3.701999999999998, -3.008666666666667]","[-1.3426666666666662, 2.236000000000004, -5.253666666666667]","[1.5513333333333357, -5.938000000000002, 8.262333333333332]","[[-0.2086666666666659, 3.701999999999998, -3.008666666666667], [-1.3426666666666662, 2.236000000000004, -5.253666666666667], [1.5513333333333357, -5.938000000000002, 8.262333333333332]]",6.97759
1,89,97,132,B,B,B,"[19.611, 70.025, 3.301]","[21.002, 74.191, 0.793]","[23.896, 66.017, 14.309]","[[19.611, 70.025, 3.301], [21.002, 74.191, 0.793], [23.896, 66.017, 14.309]]","[21.503, 70.07766666666667, 6.134333333333333]","[-1.8919999999999995, -0.052666666666667084, -2.833333333333333]","[-0.5010000000000012, 4.11333333333333, -5.341333333333333]","[2.3930000000000007, -4.060666666666677, 8.174666666666667]","[[-1.8919999999999995, -0.052666666666667084, -2.833333333333333], [-0.5010000000000012, 4.11333333333333, -5.341333333333333], [2.3930000000000007, -4.060666666666677, 8.174666666666667]]",6.53457
2,97,98,132,B,B,B,"[21.002, 74.191, 0.793]","[21.017, 73.308, -0.443]","[23.896, 66.017, 14.309]","[[21.002, 74.191, 0.793], [21.017, 73.308, -0.443], [23.896, 66.017, 14.309]]","[21.971666666666664, 71.17200000000001, 4.886333333333333]","[-0.9696666666666651, 3.0189999999999912, -4.093333333333333]","[-0.9546666666666646, 2.1359999999999957, -5.329333333333333]","[1.9243333333333368, -5.155000000000015, 9.422666666666666]","[[-0.9696666666666651, 3.0189999999999912, -4.093333333333333], [-0.9546666666666646, 2.1359999999999957, -5.329333333333333], [1.9243333333333368, -5.155000000000015, 9.422666666666666]]",7.30325
3,97,91,132,B,B,B,"[21.002, 74.191, 0.793]","[19.973, 71.736, 4.932]","[23.896, 66.017, 14.309]","[[21.002, 74.191, 0.793], [19.973, 71.736, 4.932], [23.896, 66.017, 14.309]]","[21.623666666666665, 70.64800000000001, 6.678]","[-0.6216666666666661, 3.542999999999992, -5.885]","[-1.650666666666666, 1.0879999999999939, -1.7459999999999996]","[2.2723333333333358, -4.631000000000014, 7.630999999999999]","[[-0.6216666666666661, 3.542999999999992, -5.885], [-1.650666666666666, 1.0879999999999939, -1.7459999999999996], [2.2723333333333358, -4.631000000000014, 7.630999999999999]]",6.24862
4,96,89,132,B,B,B,"[22.136, 75.657, 3.038]","[19.611, 70.025, 3.301]","[23.896, 66.017, 14.309]","[[22.136, 75.657, 3.038], [19.611, 70.025, 3.301], [23.896, 66.017, 14.309]]","[21.881, 70.56633333333333, 6.882666666666666]","[0.254999999999999, 5.090666666666664, -3.8446666666666665]","[-2.2699999999999996, -0.541333333333327, -3.581666666666666]","[2.0150000000000006, -4.549333333333337, 7.426333333333333]","[[0.254999999999999, 5.090666666666664, -3.8446666666666665], [-2.2699999999999996, -0.541333333333327, -3.581666666666666], [2.0150000000000006, -4.549333333333337, 7.426333333333333]]",6.53279


In [130]:
f1.head()

,rmsd,candidatos,cumple,candidato_clique_1,candidato_clique_2
123,0.14247,"(10, 616)",1,10,616
166,0.14225,"(10, 1085)",1,10,1085
273,0.14233,"(10, 1403)",1,10,1403
291,0.14044,"(10, 1456)",1,10,1456
308,0.11607,"(10, 1664)",1,10,1664


# Se agrega un nuevo elemento

In [220]:
def add_element_clique(df_cliques, col_candidatos, cliques, candidatos_df, number_elements_clique):
    print('numero de elementos del clique:', number_elements_clique)
    cliques_maximales = cliques[cliques.numero_elementos >= number_elements_clique].drop('numero_elementos',1).values
    set_candidatos = [df_cliques.iloc[i, range(number_elements_clique)].values for i in candidatos_df[col_candidatos].unique()]
    #conjunto de candidatos unicos
    lista_residuos = []  # aqui se guardara la lista de numero de residuo
    for candidato in set_candidatos:  # este va a cambiar cada iteracion
        for clique_maximal in cliques_maximales:
            clique_maximal = [i for i in clique_maximal if str(i) != 'nan']
            if set(candidato).issubset(clique_maximal):       # si esta en un clique maximal
                no_estan_en_clique = set(clique_maximal).difference(set(candidato))
                # obten los elementos que no estan en ese clique que si estan en el clique maximal
                for nuevo_elemento in no_estan_en_clique:
                    candidato_nuevo = candidato.copy()
                    # se genera una copia para no borrar el orginal
                    candidato_nuevo = np.append(candidato_nuevo,nuevo_elemento)
                    # se apila un elemento de los que no estan
                    if set(candidato_nuevo) not in lista_residuos:
                        lista_residuos.append(set(candidato_nuevo))
                        # si no esta en la lista pasa
    print('numero de elementos del clique ahora:', number_elements_clique+1)
    return(pd.DataFrame(lista_residuos))

In [226]:
new_df_cliques1 = add_element_clique(df_cliques1, 'candidato_clique_1',cliques1,f1,num_cliques)
new_df_cliques2 = add_element_clique(df_cliques2, 'candidato_clique_2',cliques2,f1,num_cliques)
num_cliques = num_cliques + 1

numero de elementos del clique: 4
numero de elementos del clique ahora: 5
numero de elementos del clique: 4
numero de elementos del clique ahora: 5
CPU times: user 2.68 s, sys: 12.2 ms, total: 2.69 s
Wall time: 2.64 s


In [142]:
# def get_cliques_4(df_cliques,col,cliques):
#     set_clique = [df_cliques.iloc[i, [0,1,2]].values for i in f1[col].unique()] #conjunto de candidatos unicos
#     lista_chidi = [] # aqui se guardara la lista de numero de residuo
#     cliques_maximales = cliques.drop('numero_elementos',1).values
#     for muestra in set_clique: # este va a cambiar cada iteracion
#         for i in cliques_maximales:
#             if set(muestra).issubset(i):       # si esta en un clique maximal
#                 no_estan_en_clique = set(i).difference(set(muestra)) #obten los elementos que no estan en ese clique que si estan en el clique maximal
#                 for j in no_estan_en_clique: 
#                     muestra_falsa = muestra.copy() #se genera una copia para no borrar el orginal
#                     muestra_falsa = np.append(muestra_falsa,j) #se apila un elemento de los que no estan
#                     if set(muestra_falsa) not in lista_chidi:
#                         lista_chidi.append(set(muestra_falsa)) #si no esta en la lista pasa

#     return(pd.DataFrame(lista_chidi))

In [143]:
# num_cliques = 3
# new_df_cliques1 = get_cliques_4(df_cliques1, 'candidato_clique_1',cliques1)
# # new_df_cliques2 = get_cliques_4(df_cliques2, 'candidato_clique_2',cliques2)
# # num_cliques = num_cliques + 1

# VA DE NUEZ EL CALCULO

In [227]:
def iter_rmsd(new_df_cliques1,new_df_cliques2,number_elements_clique):
    new_df_cliques1 = fc.paste_SS(ss1, new_df_cliques1, num_cliques=number_elements_clique)
    new_df_cliques2 = fc.paste_SS(ss2, new_df_cliques2, num_cliques=number_elements_clique)
    candidatos_ss = fc.compare_SS(new_df_cliques1, new_df_cliques2, num_cliques=number_elements_clique)
    df_cliques1 = fc.get_coords_clique(df_atoms1, new_df_cliques1, number_elements_clique)
    df_cliques2 = fc.get_coords_clique(df_atoms2, new_df_cliques2, number_elements_clique)
    df_cliques1 = fc.baricenter_clique(df_cliques1, number_elements_clique)
    df_cliques2 = fc.baricenter_clique(df_cliques2, number_elements_clique)
    df_cliques1 = fc.center_vectors(df_cliques1, number_elements_clique)
    df_cliques2 = fc.center_vectors(df_cliques2, number_elements_clique)
    idx_rmsd1, idx_rmsd2 = 3 * number_elements_clique, 4 * number_elements_clique + 3
    array_df_cliques1 = df_cliques1.values[:, range(idx_rmsd1, idx_rmsd2)]  # del 9 al 15
    array_df_cliques2 = df_cliques2.values[:, range(idx_rmsd1, idx_rmsd2)]
    print(len(candidatos_ss))
    df_cliques1 = fc.get_distancia_promedio(number_elements_clique, df_cliques1)
    df_cliques2 = fc.get_distancia_promedio(number_elements_clique, df_cliques2)
    array_dist_promedio1 = df_cliques1.values[:, -1]  # el ultimo valor de distancia.
    array_dist_promedio2 = df_cliques2.values[:, -1]
    limite_distancia_minima = 0.45
    if number_elements_clique == 4:
        limite_distancia_minima = 0.9
    if number_elements_clique == 5:
        limite_distancia_minima = 1.8
    if number_elements_clique == 6:
        limite_distancia_minima = 3.6
    if number_elements_clique == 7:
        limite_distancia_minima = 4.5
    if number_elements_clique == 8:
        limite_distancia_minima = 8.0

    # filtro por distancia minima
    candidatos_filter_dist = [(i, j) for i, j in candidatos_ss if (
            array_dist_promedio1[i] - array_dist_promedio2[j] >= -limite_distancia_minima) & (
                                      array_dist_promedio1[i] - array_dist_promedio2[j] <= limite_distancia_minima)]

    print('num candidatos filtro SS', len(candidatos_ss))
    print('num candidatos filtro distancia y ss', len(candidatos_filter_dist))

    p = multiprocessing.Pool(multiprocessing.cpu_count() - 1)
    restriccion_rmsd = 0.15
    if number_elements_clique == 4:
        restriccion_rmsd = 0.30
    if number_elements_clique == 5:
        restriccion_rmsd = 0.60
    if number_elements_clique == 6:
        restriccion_rmsd = 0.90
    if number_elements_clique == 7:
        restriccion_rmsd = 1.50
    if number_elements_clique == 8:
        restriccion_rmsd = 1.80

    rmsd_1 = p.map(partial(fc.calculate_rmsd_rot_trans_m,
                           array_cliques1=array_df_cliques1,
                           array_cliques2=array_df_cliques2,
                           num_cliques=number_elements_clique), candidatos_filter_dist)

    p.close()
    p.join()

    f1 = pd.DataFrame(rmsd_1)
    f1 = f1[f1[0] <= restriccion_rmsd]
    f1['candidato_clique_1'] = f1[1].str.get(0)
    f1['candidato_clique_2'] = f1[1].str.get(1)
    candidatos = f1[1].values

    time = datetime.datetime.now()

    print('numero de candidatos:', len(candidatos))
    print('tiempo pasado:', time - timenow)

    new_df_cliques1 = fc.add_element_clique(df_cliques1, 'candidato_clique_1', cliques1, f1, number_elements_clique)
    new_df_cliques2 = fc.add_element_clique(df_cliques2, 'candidato_clique_2', cliques2, f1, number_elements_clique)
    number_elements_clique = number_elements_clique + 1
    return(new_df_cliques1, new_df_cliques2, number_elements_clique)

In [ ]:
for k in range(4):
    new_df_cliques1, new_df_cliques2, number_elements_clique = iter_rmsd(
        new_df_cliques1,new_df_cliques2,number_elements_clique)

In [99]:
ahora = datetime.datetime.now()

In [101]:
print(ahora - tiempo)

0:04:32.441614
